# Just curves

In [ ]:
import holoviews as hv; hv.extension('bokeh')
hv.Curve([])

## using pd df

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import holoviews as hv; hv.extension('bokeh')
import panel as pn; pn.extension()

amp_dim = hv.Dimension("amplitude", unit="µV")
time_dim = hv.Dimension("time", unit="ms")

n_channels = 10
n_seconds = 5
total_samples = 256*n_seconds
start_datetime = datetime(2024, 1, 1)
time = np.array([start_datetime + timedelta(seconds=t) for t in np.linspace(0, n_seconds, total_samples)])

data = np.random.randn(n_channels, total_samples).cumsum(axis=1)
channels = [f"EEG {i}" for i in range(n_channels)]

df = pd.DataFrame(data.T, index=time, columns=channels)
df.index.name = 'time'

hover_tooltips=[
    ("type", "$group"),
    ("channel", "$label"),
    ("time", '@time{%H:%M:%S.%3N}'),
    ("amplitude"),
]

curves = {}
for channel_name, channel_data in df.items():
    ds = hv.Dataset((channel_data.index, channel_data, channel), [time_dim, amp_dim, "channel"])
    curve = hv.Curve(ds, time_dim, [amp_dim, "channel"], label=channel_name, group='EEG')
    curve.opts(color="black", line_width=1, subcoordinate_y=True, subcoordinate_scale=3,
              hover_tooltips = hover_tooltips)
    curves[channel_name] = curve

curves_overlay = hv.Overlay(curves, kdims="channel").opts(padding=0, aspect=2, responsive=True,show_legend=False)

curves_overlay

In [ ]:

# import h5py
import numpy as np

import holoviews as hv; hv.extension('bokeh')
from bokeh.models import HoverTool
import panel as pn; pn.extension()

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")])

n_channels = 10
n_seconds = 5
total_samples = 256*n_seconds
time = np.linspace(0, n_seconds, total_samples)
data = np.random.randn(n_channels, total_samples).cumsum(axis=1)
channels = [f"EEG {i}" for i in range(n_channels)]

channel_curves = []
for channel, channel_data in zip(channels, data):
    ds = hv.Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=channel)
    curve.opts(color="black", line_width=1, subcoordinate_y=True, subcoordinate_scale=3, tools=['hover'])
    channel_curves.append(curve)

curves = hv.Overlay(channel_curves, kdims="Channel").opts(padding=0, aspect=3, responsive=True,)

curves

# Avoid for loop, but cannot use subcoordinate_y with label being explicitly added

In [ ]:
import numpy as np
import holoviews as hv
import xarray as xr
hv.extension('bokeh')

n_channels = 10
n_seconds = 5
total_samples = 256*n_seconds

time = np.linspace(0, n_seconds, total_samples)
data = np.random.randn(n_channels, total_samples).cumsum(axis=1)
channels = [f"EEG {i}" for i in range(n_channels)]

data_xr = xr.DataArray(data, dims=['channel', 'time'], coords={'channel': channels, 'time': time}, name='value')
curves = hv.Dataset(data_xr).to(hv.Curve, 'time', 'value', 'channel').overlay('channel').opts(
    hv.opts.Curve(
        tools=['hover'],
        # subcoordinate_y=True # Currently requires a unique label per item
    ),
    hv.opts.NdOverlay(
        responsive=True,
        aspect=3,
    )
)
curves

# Avoid for loop and use channel label and group (doesn't work)

In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
hv.extension('bokeh')

n_channels = 10
n_seconds = 5
total_samples = 256 * n_seconds
groups = ['A', 'B', 'C']

time = np.linspace(0, n_seconds, total_samples)
data = np.random.randn(n_channels, total_samples).cumsum(axis=1)
channels = [f"EEG {i}" for i in range(n_channels)]

channel_groups = [groups[i % len(groups)] for i in range(n_channels)]

data_xr = xr.DataArray(
    data,
    dims=['channel', 'time'], 
    coords={
        'channel': channels, 
        'time': time,
        'group': ('channel', channel_groups)
    },
    name='value'
)

curves = hv.Dataset(data_xr).to(hv.Curve, 'time', 'value', ['channel', 'group']).overlay('channel').opts(
    hv.opts.Curve(
        tools=['hover'],
    ),
    hv.opts.NdOverlay(
        responsive=True,
        aspect=2,
    )
)

curves


In [ ]:
data_xr

In [ ]:
print(curves)

# Curves, minimap

In [ ]:
from pathlib import Path
from scipy.stats import zscore
import h5py
import numpy as np

import holoviews as hv; hv.extension('bokeh')
from holoviews.plotting.links import RangeToolLink
from holoviews.operation.datashader import rasterize
from bokeh.models import HoverTool
import panel as pn; pn.extension()

# local_data_dir = Path.home() / "data" / "ephys_sim_neuropixels"
# filename = 'ephys_sim_neuropixels_10s_384ch.h5'
# full_path = local_data_dir / filename

# f = h5py.File(full_path, "r")
 
# n_sample_chans = 40
# n_sample_times = 25000 # sampling frequency is 25 kHz
# clim_mul = 2

# time = f['timestamps'][:n_sample_times]
# data = f['recordings'][:n_sample_times,:n_sample_chans].T

# f.close()

# channels = [f'ch{i}' for i in range(n_sample_chans)]
# channels = channels[:n_sample_chans]

# main plot
hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")])

n_channels = 10 
n_seconds = 15

total_samples = 512*n_seconds
time = np.linspace(0, n_seconds, total_samples)
data = np.random.randn(n_channels, total_samples).cumsum(axis=1)
channels = [f"EEG {i}" for i in range(n_channels)]

channel_curves = []
for i, channel in enumerate(channels):
    ds = hv.Dataset((time, data[i,:], channel), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=f'{channel}')
    curve.opts(color="black", line_width=1, subcoordinate_y=True, subcoordinate_scale=3, tools=[hover])
    channel_curves.append(curve)


curves = hv.Overlay(channel_curves, kdims="Channel")

curves = curves.opts(
    xlabel="Time (s)", ylabel="Channel", show_legend=False,
    padding=0, aspect=1.5, responsive=True, shared_axes=False, framewise=False)

# minimap
y_positions = range(len(channels))
yticks = [(i, ich) for i, ich in enumerate(channels)]
z_data = zscore(data, axis=1)

minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', yticks=[yticks[0], yticks[-1]], toolbar='disable',
    height=120, responsive=True)

RangeToolLink(minimap, curves, axes=["x", "y"],
              boundsx=(.1, .3),
              boundsy=(10, 30))

(curves + minimap).cols(1)

# Sine waves, Holonote Annotator, Minimap

In [ ]:
import numpy as np
import holoviews as hv
from bokeh.models import HoverTool
from holoviews.plotting.links import RangeToolLink
from scipy.stats import zscore
from holoviews.operation.datashader import rasterize
from holonote.annotate import Annotator
import pandas as pd

hv.extension('bokeh')

N_CHANNELS = 10
N_SECONDS = 5
SAMPLING_RATE = 200
INIT_FREQ = 2  # Initial frequency in Hz
FREQ_INC = 5  # Frequency increment
AMPLITUDE = 1

# Generate time and channel labels
total_samples = N_SECONDS * SAMPLING_RATE
time = np.linspace(0, N_SECONDS, total_samples)
channels = [f'EEG {i}' for i in range(N_CHANNELS)]

# Generate sine wave data
data = np.array([AMPLITUDE * np.sin(2 * np.pi * (INIT_FREQ + i * FREQ_INC) * time)
                     for i in range(N_CHANNELS)])

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")
])

channel_curves = []
for channel, channel_data in zip(channels, data):
    ds = hv.Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=channel)
    curve.opts(
        subcoordinate_y=True, color="black", line_width=1, tools=[hover],
    )
    channel_curves.append(curve)

eeg_curves = hv.Overlay(channel_curves, kdims="Channel")

annotator = Annotator({"Time": float}, fields=["category"])
annotations_df = pd.DataFrame({'start': [1], 'end': [2], 'category': ['demo']})
annotator.define_annotations(annotations_df, Time=("start", "end"))
annotations_overlay = annotator.get_element("Time")

eeg_app = (annotations_overlay * eeg_curves).opts(
    xlabel="Time (s)", ylabel="Channel", show_legend=False, aspect=3, responsive=True,
)

y_positions = range(N_CHANNELS)
yticks = [(i , ich) for i, ich in enumerate(channels)]

z_data = zscore(data, axis=1)

minimap = rasterize(hv.Image((time, y_positions , z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))
minimap = minimap.opts(
    cmap="RdBu_r", xlabel='Time (s)', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=150, responsive=True, default_tools=[], clim=(-z_data.std(), z_data.std())
)

RangeToolLink(
    minimap, eeg_curves, axes=["x", "y"],
    boundsx=(None, 2), boundsy=(None, 6.5)
)

dashboard = (eeg_app + minimap * annotations_overlay).opts(merge_tools=False).cols(1)
dashboard


In [ ]:
import numpy as np
import holoviews as hv
from bokeh.models import HoverTool
from holoviews.plotting.links import RangeToolLink
from scipy.stats import zscore
from holoviews.operation.datashader import rasterize

hv.extension('bokeh')

N_CHANNELS = 10
N_SECONDS = 5
SAMPLING_RATE = 200
INIT_FREQ = 2  # Initial frequency in Hz
FREQ_INC = 5  # Frequency increment
AMPLITUDE = 1

# Generate time and channel labels
total_samples = N_SECONDS * SAMPLING_RATE
time = np.linspace(0, N_SECONDS, total_samples)
channels = [f'EEG {i}' for i in range(N_CHANNELS)]

# Generate sine wave data
data = np.array([AMPLITUDE * np.sin(2 * np.pi * (INIT_FREQ + i * FREQ_INC) * time)
                     for i in range(N_CHANNELS)])

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")
])

channel_curves = []
for channel, channel_data in zip(channels, data):
    ds = hv.Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=channel)
    curve.opts(
        subcoordinate_y=True, color="black", line_width=1, tools=[hover],
    )
    channel_curves.append(curve)

eeg_curves = hv.Overlay(channel_curves, kdims="Channel")

eeg_curves.opts(
    xlabel="Time (s)", ylabel="Channel", show_legend=False, aspect=3, responsive=True,
)

y_positions = range(N_CHANNELS)
yticks = [(i , ich) for i, ich in enumerate(channels)]

z_data = zscore(data, axis=1)

minimap = rasterize(hv.Image((time, y_positions , z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))
minimap = minimap.opts(
    cmap="RdBu_r", xlabel='Time (s)', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=150, responsive=True, default_tools=[], clim=(-z_data.std(), z_data.std())
)

RangeToolLink(
    minimap, eeg_curves, axes=["x", "y"],
    boundsx=(None, 2), boundsy=(None, 6.5)
)

dashboard = (eeg_curves + minimap).opts(merge_tools=False).cols(1)
dashboard


In [ ]:
import numpy as np
import holoviews as hv
from bokeh.models import HoverTool
from holoviews.plotting.links import RangeToolLink
from scipy.stats import zscore
from holoviews.operation.datashader import rasterize

hv.extension('bokeh')

# Data generation
N_CHANNELS = 10
N_SECONDS = 5
SAMPLING_RATE = 200
INIT_FREQ = 2  # Initial frequency in Hz
FREQ_INC = 5  # Frequency increment
AMPLITUDE = 1

total_samples = N_SECONDS * SAMPLING_RATE
time = np.linspace(0, N_SECONDS, total_samples)
channels = [f'EEG {i}' for i in range(N_CHANNELS)]

data = np.array([AMPLITUDE * np.sin(2 * np.pi * (INIT_FREQ + i * FREQ_INC) * time)
                     for i in range(N_CHANNELS)])

# Hover tool
hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")
])

# Create curves overlay
# TODO: utilize the input ranges
def show_curves(x_range, y_range):
    channel_curves = []
    for channel, channel_data in zip(channels, data):
        ds = hv.Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
        curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=channel).opts(
            subcoordinate_y=True, color="black", line_width=1, tools=[hover], responsive=True,
            height=400, show_legend=False,
        )
        channel_curves.append(curve)
    return hv.Overlay(channel_curves)

# RangeXY stream
range_stream = hv.streams.RangeXY(x_range=(0, 2500), y_range=(0, 1))

# DynamicMap linking show_curves to range_stream
curves = hv.DynamicMap(show_curves, streams=[range_stream])

y_positions = range(N_CHANNELS)
yticks = [(i, ich) for i, ich in enumerate(channels)]

z_data = zscore(data, axis=1)

minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")).opts(
    cmap="RdBu_r", xlabel='Time (s)', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=150, responsive=True, default_tools=[], clim=(-z_data.std(), z_data.std())
)

# Link minimap and curves
RangeToolLink(minimap, curves, axes=["x", "y"], boundsx=(None, 2), boundsy=(None, 6.5))

dashboard = (curves + minimap).opts(merge_tools=False).cols(1)
dashboard


In [ ]:
import numpy as np
import holoviews as hv
from bokeh.models import HoverTool
from holoviews.plotting.links import RangeToolLink
from scipy.stats import zscore
from holoviews.operation.datashader import rasterize

hv.extension('bokeh')

N_CHANNELS = 10
N_SECONDS = 5
SAMPLING_RATE = 200
INIT_FREQ = 2  # Initial frequency in Hz
FREQ_INC = 5  # Frequency increment
AMPLITUDE = 1

total_samples = N_SECONDS * SAMPLING_RATE
time = np.linspace(0, N_SECONDS, total_samples)
channels = [f'EEG {i}' for i in range(N_CHANNELS)]

data = np.array([AMPLITUDE * np.sin(2 * np.pi * (INIT_FREQ + i * FREQ_INC) * time)
                     for i in range(N_CHANNELS)])

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")
])

def show_curves(x_range, y_range):
    # when y_range for subcoords is fixed, we could try to also drop out of view channels 
    print(y_range) 
    if x_range is None:  # Fallback if no range is selected
        x_range = (0, N_SECONDS)
    # Calculate indices for slicing data based on x_range
    start_idx = max(int((x_range[0] / N_SECONDS) * total_samples), 0)
    end_idx = min(int((x_range[1] / N_SECONDS) * total_samples), total_samples)
    
    channel_curves = []
    for channel, channel_data in zip(channels, data):
        sliced_time = time[start_idx:end_idx]
        sliced_data = channel_data[start_idx:end_idx]
        ds = hv.Dataset((sliced_time, sliced_data, channel), ["Time", "Amplitude", "channel"])
        curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=channel).opts(
            color="black", line_width=1, tools=[hover], responsive=True,
            height=400, show_legend=False,subcoordinate_y=True,
        )
        channel_curves.append(curve)
    return hv.Overlay(channel_curves)

range_stream = hv.streams.RangeXY(x_range=(0, N_SECONDS), y_range=(0, 1))

curves = hv.DynamicMap(show_curves, streams=[range_stream])

y_positions = range(N_CHANNELS)
yticks = [(i, ich) for i, ich in enumerate(channels)]

z_data = zscore(data, axis=1)

minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")).opts(
    cmap="RdBu_r", xlabel='Time (s)', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=150, responsive=True, default_tools=[], clim=(-z_data.std(), z_data.std())
)

RangeToolLink(minimap, curves, axes=["x", "y"], boundsx=(None, 2), boundsy=(None, 6.5))

(curves + minimap).opts(merge_tools=False).cols(1)
